In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from PIL import Image

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_1 (Dropout)         (None, 1600)             

In [33]:
batch_size = 128
epochs = 2

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/2
422/422 [==============================] - 23s 53ms/step - loss: 0.7866 - accuracy: 0.7535 - val_loss: 0.0859 - val_accuracy: 0.9777
Epoch 2/2
422/422 [==============================] - 21s 49ms/step - loss: 0.1250 - accuracy: 0.9628 - val_loss: 0.0582 - val_accuracy: 0.9842


In [34]:
#save model to disk
filename = 'MNIST_model.sav'
model.save(filename)

INFO:tensorflow:Assets written to: MNIST_model.sav/assets


INFO:tensorflow:Assets written to: MNIST_model.sav/assets


In [62]:
model = keras.models.load_model(filename)
score = model.evaluate(x_test, y_test, verbose=0)

print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.05817193537950516
Test accuracy: 0.9819999933242798


In [63]:
base_model = Model(inputs=model.input, outputs=model.layers[-2].output)
base_model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1600)              0  

In [64]:
base_model.trainable = False

In [65]:
base_model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1600)              0  

In [66]:
def change_input_size(model,h,w,ch=3): 
    # from https://datascience.stackexchange.com/a/62280
    model._layers[0]._batch_input_shape = (None,h,w,ch)
    new_model = keras.models.model_from_json(model.to_json())
    new_model.summary()
    for layer,new_layer in zip(model.layers,new_model.layers):
        new_layer.set_weights(layer.get_weights())
    return new_model


In [68]:
def add_classification_head(base_model):
    
    o = keras.layers.Dense(2)(base_model.output)
    new_model = keras.Model(inputs=base_model.input, outputs=o)
    
    new_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return new_model

In [69]:
def read_images(N, num_classes, input_shape):

    #Declaring array of images 
    images = []

    #loop to import images from file directory
    for i in range (12): 
        x = np.asarray(Image.open("./bongard-problems-master/p"+str(N).zfill(3)+"/"+str(i)+".png").convert('L'))
        #print(x.shape)
        images.append(x)
        
    #Converting to numpy array
    dataset = np.array(images)
    dataset = dataset.reshape((-1, 104, 104, 1))
    

    #Y data
    yvalues = (0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1)
    
    # Scale images to the [0, 1] range
    dataset = dataset.astype("float32") / 255
    

    # convert class vectors to binary class matrices
    yvalues = keras.utils.to_categorical(yvalues, num_classes)    
    
    return dataset, yvalues

In [70]:
accuracies = []
losses = []

num_classes = 2
input_shape = (104, 104, 1)

for i in range (1, 5): 
    dataset, yvalues = read_images(i, num_classes, input_shape)
    
    new_model = change_input_size(base_model, 104, 104, 1)
    new_model2 = add_classification_head(new_model)
    
    #model = model_images(input_shape, num_classes, dataset, yvalues)
    new_model2.fit(dataset, yvalues)
    score = new_model2.evaluate(dataset, yvalues, verbose=0)
    accuracies.append(score[1])
    losses.append(score[0])
    
print(accuracies)
print(losses)

print("Average Accuracy: ", (sum(accuracies) / len(accuracies)) )
print("Average Loss: ", (sum(losses) / len(losses)) )

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 104, 104, 1)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 102, 102, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 51, 51, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 49, 49, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 36864)             0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 36864)             0  

1/1 [==============================] - 0s 369ms/step - loss: 8.1370 - accuracy: 0.5000
Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 104, 104, 1)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 102, 102, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 51, 51, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 49, 49, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 36864)             0         
_____________________________________

1/1 [==============================] - 1s 516ms/step - loss: 4.5117 - accuracy: 0.5833
Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 104, 104, 1)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 102, 102, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 51, 51, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 49, 49, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 36864)             0         
_____________________________________

1/1 [==============================] - 0s 360ms/step - loss: 3.2822 - accuracy: 0.5833
Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 104, 104, 1)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 102, 102, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 51, 51, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 49, 49, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 36864)             0         
_____________________________________

1/1 [==============================] - 0s 406ms/step - loss: 7.0037 - accuracy: 0.2500


[0.5, 0.5, 0.5, 0.5]
[5.853809356689453, 0.6605543494224548, 6.984872341156006, 8.05904769897461]
Average Accuracy:  0.5
Average Loss:  5.389570936560631
